In [ ]:
#doing the PCA for Control V.S. Alzeimers

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import mne
import entropy as ent
import neurokit2 as nk
from scipy.signal import welch
from nolds import lyap_r
import antropy as ant
import joblib

In [39]:
#loading in the dataframs 

In [63]:
alz_df = pd.read_csv("Alzeimers.csv")
alz_df.set_index(["Subject", "Channel", "Band"], inplace=True)  # Restore MultiIndex manually

control_df = pd.read_csv("control.csv")
control_df.set_index(["Subject", "Channel", "Band"], inplace=True)  

In [65]:
print(type(alz_df.loc[('sub-001', 'Fp1', 'Delta'), "Window Avg"]))


<class 'str'>


In [66]:
import ast

def convert_str_to_list(val):
    if isinstance(val, str):  # Check if value is a string
        return ast.literal_eval(val)  # Convert string to list
    return val  # Otherwise, keep as is

# Apply conversion to "Window Avg"
for 
alz_df["Window Avg"] = alz_df["Window Avg"].apply(convert_str_to_list)
control_df["Window Avg"] = control_df["Window Avg"].apply(convert_str_to_list)


ValueError: malformed node or string: <ast.Call object at 0x176fda130>

In [64]:
print(alz_df.loc[('sub-001', 'Fp1', 'Delta'), "Window Avg"][0])
print(alz_df.loc[('sub-001', 'Fp1', 'Delta'), "Window Avg"])

[
[np.float64(5.607240819161222e-07), np.float64(2.136206557103639e-07), np.float64(1.4815955569749812e-07), np.float64(1.1329436172684679e-07), np.float64(7.296527914043617e-08), np.float64(6.059866918225963e-08), np.float64(1.0361302410499309e-07), np.float64(2.3086230197239276e-07), np.float64(1.0552505598702842e-07), np.float64(8.896397838702983e-08), np.float64(6.57345729309926e-08), np.float64(6.71105777754649e-08), np.float64(1.6631586295502842e-07), np.float64(1.5266041712409792e-07), np.float64(5.313955773328878e-07), np.float64(6.700614808615797e-07), np.float64(3.464982345643528e-07), np.float64(1.0609118614977464e-06), np.float64(6.268996717597441e-07), np.float64(9.355173677896289e-08), np.float64(1.194314537437679e-07), np.float64(2.9432058402460086e-07), np.float64(3.0417203518634765e-07), np.float64(1.0967247840000785e-07), np.float64(9.184340855409266e-08), np.float64(7.298744113820288e-08), np.float64(6.505383470440462e-08), np.float64(1.0417842890080257e-07), np.floa

In [41]:
# **I don't think this is needed **
# participantsInfo = pd.read_table('./ds004504/participants.tsv')
# #Getting all the groups
# A_sub = participantsInfo[participantsInfo["Group"] == "A"]["participant_id"].tolist()
# C_sub = participantsInfo[participantsInfo["Group"] == "C"]["participant_id"].tolist()
# D_sub = participantsInfo[participantsInfo["Group"] == "F"]["participant_id"].tolist()

In [42]:
# Print Alzheimer's preview
print(f"Alzheimer's preview for sub-001")
print(alz_df.loc['sub-001']) 
print(f"Control's preview for sub-037")
print(control_df.loc['sub-037'])
# # Print Control preview
# print(f"Control preview {C_sub[0]}")
# print(groups["Control"].loc[C_sub[0]])  

# # Print Dementia preview
# print(f"Dementia preview {D_sub[0]}")
# print(groups["Dementia"].loc[D_sub[0]])  

Alzheimer's preview for sub-001
                                                      Window Avg
Channel Band                                                    
Fp1     Delta  [np.float64(5.607240819161222e-07), np.float64...
        Theta  [np.float64(1.4290601340850907e-08), np.float6...
        Alpha  [np.float64(2.877208402057422e-09), np.float64...
        Beta   [np.float64(5.046764330104262e-10), np.float64...
Fp2     Delta  [np.float64(5.465781151565962e-07), np.float64...
...                                                          ...
Cz      Beta   [np.float64(3.205899994916665e-10), np.float64...
Pz      Delta  [np.float64(1.5562185234722875e-07), np.float6...
        Theta  [np.float64(4.1626502960822e-09), np.float64(7...
        Alpha  [np.float64(1.2805820794756784e-09), np.float6...
        Beta   [np.float64(3.338654724073159e-10), np.float64...

[76 rows x 1 columns]
Control's preview for sub-037
                                                      Window Avg
Chann

In [43]:
# Creating training and testing data. Going to set aside 2 subjects for each group 


In [44]:
# Get unique subject IDs
alz_subjects = alz_df.index.get_level_values("Subject").unique()
control_subjects = control_df.index.get_level_values("Subject").unique()

# Select the last 3 subjects for testing
test_alz_subjects = alz_subjects[-3:]  # Last 3 subjects
test_control_subjects = control_subjects[-3:]  # Last 3 subjects

# Select training subjects (all except the last 3)
train_alz_subjects = alz_subjects[:-3]
train_control_subjects = control_subjects[:-3]

# Split DataFrames
alz_train_df = alz_df.loc[train_alz_subjects]
alz_test_df = alz_df.loc[test_alz_subjects]

control_train_df = control_df.loc[train_control_subjects]
control_test_df = control_df.loc[test_control_subjects]

# Print confirmation
print("Train Alzheimer's subjects:", train_alz_subjects.tolist())
print("Test Alzheimer's subjects:", test_alz_subjects.tolist())
print("*"*64)
print("Train Control subjects:", train_control_subjects.tolist())
print("Test Control subjects:", test_control_subjects.tolist())


Train Alzheimer's subjects: ['sub-001', 'sub-002', 'sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-009', 'sub-010', 'sub-011', 'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 'sub-017', 'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032', 'sub-033']
Test Alzheimer's subjects: ['sub-034', 'sub-035', 'sub-036']
****************************************************************
Train Control subjects: ['sub-037', 'sub-038', 'sub-039', 'sub-040', 'sub-041', 'sub-042', 'sub-043', 'sub-044', 'sub-045', 'sub-046', 'sub-047', 'sub-048', 'sub-049', 'sub-050', 'sub-051', 'sub-052', 'sub-053', 'sub-054', 'sub-055', 'sub-056', 'sub-057', 'sub-058', 'sub-059', 'sub-060', 'sub-061', 'sub-062']
Test Control subjects: ['sub-063', 'sub-064', 'sub-065']


In [48]:
# Remove subject from index, but keep channel and band
alz_combined = alz_train_df.reset_index(level="Subject", drop=True)
control_combined = control_train_df.reset_index(level="Subject", drop=True)


# Print results
print(alz_combined.head())
print(alz_combined.head())
print(control_combined)
print(control_combined) #check that all the channels are here , looks like they got moved

                                                      Window Avg
Channel Band                                                    
Fp1     Delta  [np.float64(5.607240819161222e-07), np.float64...
        Theta  [np.float64(1.4290601340850907e-08), np.float6...
        Alpha  [np.float64(2.877208402057422e-09), np.float64...
        Beta   [np.float64(5.046764330104262e-10), np.float64...
Fp2     Delta  [np.float64(5.465781151565962e-07), np.float64...
                                                      Window Avg
Channel Band                                                    
Fp1     Delta  [np.float64(5.607240819161222e-07), np.float64...
        Theta  [np.float64(1.4290601340850907e-08), np.float6...
        Alpha  [np.float64(2.877208402057422e-09), np.float64...
        Beta   [np.float64(5.046764330104262e-10), np.float64...
Fp2     Delta  [np.float64(5.465781151565962e-07), np.float64...
                                                      Window Avg
Channel Band             

In [55]:
alz_combined["Window Avg"] = alz_combined["Window Avg"].apply(lambda x: float(x) if isinstance(x, (list, np.ndarray)) and len(x) == 1 else x)
control_combined["Window Avg"] = control_combined["Window Avg"].apply(lambda x: float(x) if isinstance(x, (list, np.ndarray)) and len(x) == 1 else x)

print(alz_combined.head())
print(alz_combined.head())
print(control_combined)
print(control_combined) #check that all the channels are here , looks like they got moved

                                                      Window Avg
Channel Band                                                    
Fp1     Delta  [np.float64(5.607240819161222e-07), np.float64...
        Theta  [np.float64(1.4290601340850907e-08), np.float6...
        Alpha  [np.float64(2.877208402057422e-09), np.float64...
        Beta   [np.float64(5.046764330104262e-10), np.float64...
Fp2     Delta  [np.float64(5.465781151565962e-07), np.float64...
                                                      Window Avg
Channel Band                                                    
Fp1     Delta  [np.float64(5.607240819161222e-07), np.float64...
        Theta  [np.float64(1.4290601340850907e-08), np.float6...
        Alpha  [np.float64(2.877208402057422e-09), np.float64...
        Beta   [np.float64(5.046764330104262e-10), np.float64...
Fp2     Delta  [np.float64(5.465781151565962e-07), np.float64...
                                                      Window Avg
Channel Band             

In [56]:
# go to this Step 1: Ensure "Window Avg" is Numeric Before we scale, we need to ensure that "Window Avg" is stored as numeric values.
# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the Alzheimer's data
alz_combined["Window Avg"] = scaler.fit_transform(alz_combined[["Window Avg"]])

# Transform (but not fit) the Control data with the same scaler
control_combined["Window Avg"] = scaler.transform(control_combined[["Window Avg"]])

# Check results
print("Scaled Alzheimer's Data:\n", alz_combined.head())
print("Scaled Control Data:\n", control_combined.head())

ValueError: could not convert string to float: '[np.float64(5.607240819161222e-07), np.float64(2.136206557103639e-07), np.float64(1.4815955569749812e-07), np.float64(1.1329436172684679e-07), np.float64(7.296527914043617e-08), np.float64(6.059866918225963e-08), np.float64(1.0361302410499309e-07), np.float64(2.3086230197239276e-07), np.float64(1.0552505598702842e-07), np.float64(8.896397838702983e-08), np.float64(6.57345729309926e-08), np.float64(6.71105777754649e-08), np.float64(1.6631586295502842e-07), np.float64(1.5266041712409792e-07), np.float64(5.313955773328878e-07), np.float64(6.700614808615797e-07), np.float64(3.464982345643528e-07), np.float64(1.0609118614977464e-06), np.float64(6.268996717597441e-07), np.float64(9.355173677896289e-08), np.float64(1.194314537437679e-07), np.float64(2.9432058402460086e-07), np.float64(3.0417203518634765e-07), np.float64(1.0967247840000785e-07), np.float64(9.184340855409266e-08), np.float64(7.298744113820288e-08), np.float64(6.505383470440462e-08